In [363]:
import json
import cfscrape
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
from fake_useragent import UserAgent
useragent = UserAgent() 

In [364]:
useragent.random 

'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1464.0 Safari/537.36'

In [365]:
 yandex_all_id = 59452068765, 1230403863, 135157708122, 13084226257, 122953255163, 1166206684, 1170778562, 1080903330, 1315558370, 1168675286, 1036543306, 5651209305,66697421636,1199923913,1146856379,1752726383,68938452823,1136669337,48923682156,1084785161,106396488794,1169237983,1171367346,1226210503,244588988530,1249340350,1109319679,129281957669,222501363394,95798602015,1808210910,74755559887,39505328349,58618253938,181349390806,222108751806,1004521162,238385424377,1178994977,1809097327,225447574680,1200427674,1142895335,46866695409,1261132791,1780139580,227594122810,57361398610,1360515342,1187228400,205284667443,175499965097,1234696811,1219578186,1243520916,1137893690,1163323346,1395617057,1156420979,1261357675,1137049971,1503884741,147075068049,1173855410,1209203365,1084635139,1357472078,111881168077,1737906159,1159394297,1216178581,1092610521,1218514905

In [366]:
#yandex_all_id = 1199923913,1146856379,1752726383,68938452823,1136669337,48923682156,1084785161,106396488794,1169237983,1171367346,244588988530,95798602015,1808210910,74755559887,222108751806,1004521162,238385424377,1178994977,1809097327


In [367]:
#yandex_all_id = 1146856379,1752726383,68938452823,1136669337,48923682156,1084785161,106396488794,1169237983,1171367346


In [368]:
Data_id = pd.DataFrame((yandex_all_id), columns = ['id'])

In [369]:
Data_id.head()

,id
0,59452068765
1,1230403863
2,135157708122
3,13084226257
4,122953255163


In [370]:
len(Data_id)

73

In [371]:
#'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'

In [372]:
estimates_all = pd.DataFrame()

In [373]:
for i in range(len(Data_id)):
    yandex_id = str(Data_id.iloc[i]['id'])
    date_object = time.strftime("%d.%m.%Y")
    time_object = time.strftime("%H:%M:%S")
    ya = pd.DataFrame({"date": date_object,"time":time_object, "id": yandex_id}, index = [1])
    ya['key'] = 1
    def get_count_star(review_stars):
        star_count = 0
        for review_star in review_stars:
            if '_empty' in review_star.get('class'):
                continue
            elif '_half' in review_star.get('class'):
                star_count = star_count + 0.5
            else:
                star_count = star_count + 1
        return star_count

    def get_session():
        session = requests.Session()
        session.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
            'upgrade-insecure-requests': '1',
            #'user-agent': useragent.random,
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
            'viewport-width': '1920'
        }
        return cfscrape.create_scraper(sess=session)

    def parsing_data(data_result, r):
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            reviews = soup.find('div', {"data-chunk": "reviews"})
        except:
            return None
        try:
            summary_rating__main = reviews.find('div', {"class": "business-summary-rating__main-rating"})
        except:
            summary_rating__main = None
        try:
            rank_summary_rating__main = summary_rating__main.find('span',{"class": "business-summary-rating-badge-view__rating"}).text
        except:
            rank_summary_rating__main = None
        try:
            count_summary_rating__main = summary_rating__main.find('div', {
                "class": "business-summary-rating-badge-view__rating-count"}).text
        except:
            count_summary_rating__main = None
        try:
            stars_summary_rating__main = get_count_star(
                summary_rating__main.find('div', {"class": "business-rating-badge-view__stars"}))
        except:
            stars_summary_rating__main = None
        try:
            address = soup.find('meta', {"itemprop": "address"}).get('content')
        except:
            address = None
        data_result['company_info'] = {
            'company_rating': rank_summary_rating__main,
            'company_count_reviews': count_summary_rating__main,
            'company_count_stars': stars_summary_rating__main,
            'company_address': address
        }
        
        return data_result
    
    url = 'https://yandex.ru/maps/org/letay/' + yandex_id
    session = get_session()
    r = session.get(url)
    data_result = {
        'company_info': {}
    }
    data_result
    data_result = parsing_data(data_result, r)
    df = pd.DataFrame(data_result)
    addr = df.loc['company_address']['company_info']
    while addr == None:
        time_new = time.strftime("%H:%M:%S")
        print(time_new,' New query for ',yandex_id)
        time.sleep(300)
        session = get_session()
        r = session.get(url)
        data_result = {
            'company_info': {}
        }
        data_result
        data_result = parsing_data(data_result, r)
        df = pd.DataFrame(data_result)
        addr = df.loc['company_address']['company_info']
    df = df.transpose()   
    if len(data_result['company_info']) == 0:
        data_result = None
    with open('company_reviews.json', 'w', encoding='utf-8') as f:
        json.dump(data_result, f, ensure_ascii=False, indent=4)
    print(i,' ',time_object,' ',yandex_id,' ',url,' - Parsing Success')
    df['key'] = 1
    df['url'] = url
    df = ya.merge(df, on = 'key', indicator=True)
    estimates_all = pd.concat([estimates_all,df])
    time.sleep(7+i)

0   07:09:54   59452068765   https://yandex.ru/maps/org/letay/59452068765  - Parsing Success
1   07:10:01   1230403863   https://yandex.ru/maps/org/letay/1230403863  - Parsing Success
2   07:10:10   135157708122   https://yandex.ru/maps/org/letay/135157708122  - Parsing Success
3   07:10:19   13084226257   https://yandex.ru/maps/org/letay/13084226257  - Parsing Success
4   07:10:30   122953255163   https://yandex.ru/maps/org/letay/122953255163  - Parsing Success
5   07:10:41   1166206684   https://yandex.ru/maps/org/letay/1166206684  - Parsing Success
6   07:10:54   1170778562   https://yandex.ru/maps/org/letay/1170778562  - Parsing Success
7   07:11:07   1080903330   https://yandex.ru/maps/org/letay/1080903330  - Parsing Success
8   07:11:22   1315558370   https://yandex.ru/maps/org/letay/1315558370  - Parsing Success
9   07:11:37   1168675286   https://yandex.ru/maps/org/letay/1168675286  - Parsing Success
10   07:11:54   1036543306   https://yandex.ru/maps/org/letay/1036543306  - Pa

In [374]:
estimates_all

,date,time,id,key,company_address,company_count_reviews,company_count_stars,company_rating,url,_merge
0,02.11.2022,07:09:54,59452068765,1,"Республика Татарстан, Заинск, Казанская улица,...",None,None,None,https://yandex.ru/maps/org/letay/59452068765,both
0,02.11.2022,07:10:01,1230403863,1,"Республика Татарстан, Альметьевск, улица Маяко...",343 оценки,3.5,"Рейтинг3,3",https://yandex.ru/maps/org/letay/1230403863,both
0,02.11.2022,07:10:10,135157708122,1,"Республика Татарстан, Нижнекамск, улица Баки У...",111 оценок,1.5,"Рейтинг1,6",https://yandex.ru/maps/org/letay/135157708122,both
0,02.11.2022,07:10:19,13084226257,1,"Республика Татарстан, Мамадыш, улица Тукая, 32",None,None,None,https://yandex.ru/maps/org/letay/13084226257,both
0,02.11.2022,07:10:30,122953255163,1,"Республика Татарстан, посёлок городского типа ...",None,None,None,https://yandex.ru/maps/org/letay/122953255163,both
...,...,...,...,...,...,...,...,...,...,...
0,02.11.2022,08:51:20,1737906159,1,"Республика Татарстан, Верхнеуслонский район, И...",41 оценка,4,"Рейтинг3,9",https://yandex.ru/maps/org/letay/1737906159,both
0,02.11.2022,08:52:36,1159394297,1,"Республика Татарстан, посёлок городского типа ...",80 оценок,4,"Рейтинг4,0",https://yandex.ru/maps/org/letay/1159394297,both
0,02.11.2022,08:53:52,1216178581,1,"Республика Татарстан, Буинск, улица Розы Люксе...",127 оценок,4.5,"Рейтинг4,3",https://yandex.ru/maps/org/letay/1216178581,both
0,02.11.2022,08:55:10,1092610521,1,"Республика Татарстан, Зеленодольск, улица Карл...",200 оценок,3,"Рейтинг2,8",https://yandex.ru/maps/org/letay/1092610521,both


In [375]:
day = time.strftime("%d%m%Y_%H%M")

In [376]:
estimates_all.to_excel('D:\\estimates\estimates_all_'+day+'.xlsx',index = False)